# Decision tree demonstration

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import tree

## Data

In [ ]:
titanic = sns.load_dataset("titanic")
titanic.head()

In [ ]:
titanic = titanic.dropna()
X, y = titanic[["sex", "pclass", "sibsp"]].copy(), titanic.survived
X["sex"] = (X["sex"] == "female").astype(int)
pd.concat([X, y], axis=1)

nový exmaple
určit, jak bude klasifikovaný
jak by musel bypadat example, aby padl to XY


## Building a decision tree

We aim to "explain" the data by subdividing the data in to smaller parts that have more homogeneous labels. Meaning this particular group defined by subdivisions, e.g., woman from 1st class, mostly survived. To do that we need some way of measuring the homogeneity of labels. This can be done using entropy that measures amount of information (or randomness) in the data. It is defined as 

$$ H(X) = - \sum_{k} p_{k} log(p_{k}) $$

where $X$ is the data and $p_{k}$ is the relative frequency (or probability) of label $k$ in the $X$.

We need frequency of labels,

In [ ]:
values = y.value_counts()
values

make them relative,

In [ ]:
p0 = values[0] / len(y)
p1 = values[1] / len(y)
p0, p1

and sum up.

In [ ]:
h = -(p0 * np.log2(p0) + p1 * np.log2(p1))
h

Let's make a function from this code.

In [ ]:
def entropy(y):
    probabilities = y.value_counts() / len(y)
    return -np.sum(probabilities * np.log2(probabilities))

In [ ]:
entropy(y)

Now we know how to measure homogeneity of labels but we need to decide on how to split the data. One way is to measure change in the entropy before and after splitting. This is called information gain and it is defined as

$$ Gain(X, F) = Entropy(X) - \sum_{v \in values(F)} \frac{|X_v|}{|X|} Entropy(X_v) $$

where $X$ is the data, $F$ is the feature by which value we will split the data, and $X_v$ is a subset of the data with feature $F$ having value $v$.

Let's compute information gain for feature `sex` that has two possible values.

In [ ]:
y_sex_0 = y[X["sex"] == 0]
y_sex_1 = y[X["sex"] == 1]

In [ ]:
entropy(y_sex_0), entropy(y_sex_1)

In [ ]:
len(y_sex_0) / len(y) * entropy(y_sex_0), len(y_sex_1) / len(y) * entropy(y_sex_1)

In [ ]:
(len(y_sex_0) / len(y) * entropy(y_sex_0)) + (len(y_sex_1) / len(y) * entropy(y_sex_1))

In [ ]:
entropy(y) - (
    (len(y_sex_0) / len(y) * entropy(y_sex_0))
    + (len(y_sex_1) / len(y) * entropy(y_sex_1))
)

Let's make a function from this code.

In [ ]:
def info_gain(X, y, feature):
    feature_values = X[feature].unique()
    splited_y = [y[X[feature] == value] for value in feature_values]
    return entropy(y) - sum(
        len(subset_y) / len(y) * entropy(subset_y) for subset_y in splited_y
    )

In [ ]:
info_gain(X, y, "sex")

Now we can asses how much information we gain by splitting by each feature. All that remains is to select the one that has maximal informational gain a use it as for splitting.

In [ ]:
info_gain(X, y, "sex"), info_gain(X, y, "pclass"), info_gain(X, y, "sibsp")

Let's check that the scikit-learn's implementation also select `sex` in the first split.

In [ ]:
dtc = tree.DecisionTreeClassifier(criterion="entropy", max_depth=1)
dtc.fit(X, y)

plt.figure(dpi=150)
tree.plot_tree(dtc, filled=True, rounded=True, feature_names=X.columns)
pass

Great, let's continue recursively splitting leafs.

In [ ]:
X_sex0, y_sex0 = X[X.sex == 0], y[X.sex == 0]
X_sex1, y_sex1 = X[X.sex == 1], y[X.sex == 1]

In [ ]:
info_gain(X_sex0, y_sex0, "pclass"), info_gain(X_sex0, y_sex0, "sibsp")

In [ ]:
info_gain(X_sex1, y_sex1, "pclass"), info_gain(X_sex1, y_sex1, "sibsp")

We choose `sbisp` to split one leaf and `pclass` to split the other. But these features are not binary, i.e. they have more than two values. We could either add multiple new nodes one for each feature value or make a split in form of values $\leq$ threshold and values $>$ threshold. The latter is what scikit-learn's implementation is doing, so we need to find the split thresholds. Basically, we need to check every possible split between minimal and maximal feature value and check its information gain.

In [ ]:
sibsp_values = sorted(X_sex0.sibsp.unique())
pclass_values = sorted(X_sex1.pclass.unique())

In [ ]:
print("sibsp")
for i in range(len(sibsp_values) - 1):
    threshold = (sibsp_values[i] + sibsp_values[i + 1]) / 2
    y_leq = y_sex0[X_sex0.sibsp <= threshold]
    y_ge = y_sex0[X_sex0.sibsp > threshold]
    gain = entropy(y_sex0) - (
        len(y_leq) / len(y_sex0) * entropy(y_leq)
        + len(y_ge) / len(y_sex0) * entropy(y_ge)
    )
    print(f"threshold: {threshold}, info gain: {gain}")

print("pclass")
for i in range(len(pclass_values) - 1):
    threshold = (pclass_values[i] + pclass_values[i + 1]) / 2
    y_leq = y_sex1[X_sex1.pclass <= threshold]
    y_ge = y_sex1[X_sex1.pclass > threshold]
    gain = entropy(y_sex1) - (
        len(y_leq) / len(y_sex1) * entropy(y_leq)
        + len(y_ge) / len(y_sex1) * entropy(y_ge)
    )
    print(f"threshold: {threshold}, info gain: {gain}")

So we split on `sibsp` at $0.5$ and on `pclass` on $2.5$. Let's check with scikit-learn's implementation.

In [ ]:
dtc = tree.DecisionTreeClassifier(criterion="entropy", max_depth=2)
dtc.fit(X, y)

plt.figure(dpi=200)
tree.plot_tree(
    dtc,
    filled=True,
    rounded=True,
    node_ids=True,
    feature_names=X.columns,
    class_names=("Died", "Survived"),
)
pass

## Classifying new data
Let's say we have following new data to classify.

In [ ]:
new_X = pd.DataFrame({"sex": [0, 1, 1], "pclass": [1, 2, 3], "sibsp": [0, 1, 3],})
new_X

Who will these examples be classified?

In [ ]:
dtc.predict(new_X)

In [ ]:
dtc.decision_path(new_X).toarray()

## Final notes

### Impurity measure
By default, the label impurity measure in scikit-learn's implementation is Gini index and not entropy. They are fairly similar and Gini is defined as

$$ H(X) = \sum_{k} p_{k}(1-p_{k}) $$

where, again, $X$ is the data and $p_k$ is relative proportion of label $k$. The main advantage of Gini over entropy is not biased towards features with many distinct values.

### Pruning
The presented tree building schema is pretty much ID3 algorithm. It is known to over-fit the data a lot. Most library implementations of decision trees are either C4.5 or CART algorithms that are based on ID3 and add few additional ideas. One of them is pruning. Pruning results in some nodes not being split if they do not significantly increase precision (pre-pruning) and some node being collapsed back into one node if the prediction accuracy is not affected much (post-pruning).